# Portfolio Management with Amazon SageMaker RL


Portfolio management is the process of constant redistribution of a capital into a set of different financial assets. Given the historic prices of a list of stocks and current portfolio allocation, the goal is to maximize the return while restraining the risk. In this demo, we use a reinforcement learning framework to manage the portfolio by continuously reallocating several stocks. Based on the setup in [1], we use a tensor input constructed from historical price data, and then apply an actor-critic policy gradient algorithm to accommodate the continuous actions (reallocations). The customized environment is constructed using Open AI Gym and the RL agents are trained using Amazon SageMaker.  

[1] Jiang, Zhengyao, Dixing Xu, and Jinjun Liang. "[A deep reinforcement learning framework for the financial portfolio management problem." arXiv preprint arXiv:1706.10059 (2017)](https://arxiv.org/abs/1706.10059).

## Problem Statement

We start with $m$ preselected stocks. Without loss of generality, the total investment value is set as 1 dollar at the initial timestamp. At timestamp $t$, letting $v_{m,t}$ denote the closing price of stock $m$, the *price relative vector* is defined as 
$$ y_t = ( 1, \frac{v_{1,t}}{v_{1,t-1}}, \frac{v_{2,t}}{v_{2,t-1}}, \dots, \frac{v_{m,t}}{v_{m,t-1}} ). $$
The first element corresponds to the cash we maintain. The cash value doesn't change along time so it is always 1. During training, the investment redistribution at step $t$ is characterized by the portfolio weight vector $\mathbf{\omega} = (\omega_{0,t}, \omega_{1,t}, \dots, \omega_{m,t})$. 

1. *Objective:*
The portfolio consists of a group of stocks. We aim to maximize the portfolio value by adjusting the weights of each stock and reallocating the portfolio at the end of each day.

2. *Environment:*
Custom developed environment using Gym.

3. *States:*
Portfolio weight vector from last trading day $\omega_{t-1}$. Historic price tensor constructed using close, open, high, low prices of each stock. For more details, please refer to [1].

4. *Actions:*
New weight vector $\omega_{t}$ satisfying $\sum_{i=0}^{m}\omega_{i,t}=1$.

5. *Reward:* 
Average logarithmic cumulated return. Consider a trading cost factor $\mu$, the average logarithmic cumulated return after timestamp $T$ is $$ R := \frac{1}{T} \sum_{t=1}^{T+1} \ln(\mu_{t}y_{t}\cdot\omega_{t-1}).$$
We use the maximum rate at Poloniex and set $\mu=0.25\%$.


## Dataset

In this notebook, we use the dataset generated by [Chi Zhang](https://github.com/vermouth1992/drl-portfolio-management/tree/master/src/utils/datasets). It contains the historic price of 16 target stocks from NASDAQ100, including open, close, high and low prices from 2012-08-13 to 2017-08-11. Specifically, those stocks are: “AAPL”, “ATVI”, “CMCSA”, “COST”, “CSX”, “DISH”, “EA”, “EBAY”, “FB”, “GOOGL”, “HAS”, “ILMN”, “INTC”, “MAR”, “REGN” and “SBUX”.


### Dataset License
This dataset is licensed under a MIT License.

Copyright (c) 2017 Chi Zhang

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

## Using reinforcement learning on Amazon SageMaker RL

Amazon SageMaker RL allows you to train your RL agents using an on-demand and fully managed infrastructure. You do not have to worry about setting up your machines with the RL toolkits and deep learning frameworks as there are pre-built RL environments. You can easily switch between many different machines setup for you, including powerful GPU machines that give a big speedup. You can also choose to use multiple machines in a cluster to further speedup training, often necessary for production level loads.



## Pre-requisites

### Roles and permissions

To get started, we'll import the Python libraries we need, set up the environment with a few prerequisites for permissions and configurations.

In [1]:
import sagemaker
import boto3
import sys
import os
import glob
import re
import subprocess
from IPython.display import HTML
import time
from time import gmtime, strftime
sys.path.append("common")
from misc import get_execution_role, wait_for_s3_object
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework

### Steup S3 buckets

Set up the linkage and authentication to the S3 bucket that you want to use for checkpoint and the metadata. 

In [2]:
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()  
s3_output_path = 's3://{}/'.format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))

S3 bucket path: s3://sagemaker-eu-central-1-415877977751/


### Define Variables 

We define variables such as the job prefix for the training jobs.

In [3]:
# create unique job name 
job_name_prefix = 'rl-macroeconomic'

### Configure settings

You can run your RL training jobs on a SageMaker notebook instance or on your own machine. In both of these scenarios, you can run the following in either `local` or `SageMaker` modes. The `local` mode uses the SageMaker Python SDK to run your code in a local container before deploying to SageMaker. This can speed up iterative testing and debugging while using the same familiar Python SDK interface. You just need to set `local_mode = True`.

In [4]:
# run in local mode?
local_mode = False

### Create an IAM role
Either get the execution role when running from a SageMaker notebook `role = sagemaker.get_execution_role()` or, when running from local machine, use utils method `role = get_execution_role()` to create an execution role.

In [5]:
try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role()

print("Using IAM role arn: {}".format(role))

Couldn't call 'get_role' to get Role ARN from role name bluhmben to get Role path.


Using IAM role arn: arn:aws:iam::415877977751:role/sagemaker


### Install docker for `local` mode

In order to work in `local` mode, you need to have docker installed. When running from you local machine, please make sure that you have docker or docker-compose (for local CPU machines) and nvidia-docker (for local GPU machines) installed. Alternatively, when running from a SageMaker notebook instance, you can simply run the following script to install dependenceis.

Note, you can only run a single local notebook at one time.

In [ ]:
# Run on SageMaker notebook instance
if local_mode:
    !/bin/bash ./common/setup.sh

## Set up the environment

The environment is defined in a Python file called `portfolio_env.py` and the file is uploaded on `/src` directory. 

The environment also implements the `init()`, `step()` and `reset()` functions that describe how the environment behaves. This is consistent with Open AI Gym interfaces for defining an environment. 


1. init() - initialize the environment in a pre-defined state
2. step() - take an action on the environment
3. reset()- restart the environment on a new episode
4. [if applicable] render() - get a rendered image of the environment in its current state

In [6]:
!pygmentize src/macroeconomic_env.py

import gym
import gym.spaces
import random
import math
import csv
import numpy as np
from scipy.stats import norm


class MacroeconomicEnv(gym.Env):
    """
    An environment for optimal consumption/savings policy in a macroeconomic life-cycle model.
    Based on [Bluhm 2020](...)
    """

    def __init__(self, **config):
        config_defaults = {
            # Parameter choice based on https://www.sas.upenn.edu/~jesusfv/Guide_Parallel.pdf
            # https://github.com/davidzarruk/Parallel_Computing
            "nx": 5,    # Grid length for savings
            "xmin": 0.1,    # Lower bound for savings
            "xmax": 4.0,           # Upper bound for savings
            "ne": 15,      # Number of states (labor income shocks)
            "ssigma_eps": 0.02058,   # Variance of discrete markov chain process
            "llambda_eps": 0.99,      # Autoregressive parameter of markov chain process
            "m": 1.5,  # Grid width of markov chain process
            "ssigma": 2, 

## Configure the presets for RL algorithm 

The presets that configure the RL training jobs are defined in the `preset-portfolio-management-clippedppo.py` file which is also uploaded on the `/src` directory. Using the preset file, you can define agent parameters to select the specific agent algorithm. You can also set the environment parameters, define the schedule and visualization parameters, and define the graph manager. The schedule presets will define the number of heat up steps, periodic evaluation steps, training steps between evaluations.

These can be overridden at runtime by specifying the `RLCOACH_PRESET` hyperparameter. Additionally, it can be used to define custom hyperparameters. 


In [7]:
!pygmentize src/preset-macroeconomic-clippedppo.py

from rl_coach.agents.clipped_ppo_agent import ClippedPPOAgentParameters
from rl_coach.architectures.layers import Dense, Conv2d
from rl_coach.base_parameters import VisualizationParameters, PresetValidationParameters
from rl_coach.base_parameters import MiddlewareScheme, DistributedCoachSynchronizationType, EmbedderScheme
from rl_coach.core_types import TrainingSteps, EnvironmentEpisodes, EnvironmentSteps, RunPhase
from rl_coach.environments.gym_environment import GymVectorEnvironment, ObservationSpaceType
from rl_coach.exploration_policies.e_greedy import EGreedyParameters
from rl_coach.graph_managers.basic_rl_graph_manager import BasicRLGraphManager
from rl_coach.graph_managers.graph_manager import ScheduleParameters
from rl_coach.schedules import LinearSchedule

####################
# Graph Scheduling #
####################

schedule_params = ScheduleParameters()
schedule_params.improve_steps = TrainingSteps(int(50000))
schedule_params.steps_between_evaluation_periods = EnvironmentS

## Write the Training Code 

The training code is written in the file “train-coach.py” which is uploaded in the /src directory. 
First import the environment files and the preset files, and then define the `main()` function. 

In [8]:
!pygmentize src/train-coach.py

from sagemaker_rl.coach_launcher import SageMakerCoachPresetLauncher


class MyLauncher(SageMakerCoachPresetLauncher):

    def default_preset_name(self):
        """This points to a .py file that configures everything about the RL job.
        It can be overridden at runtime by specifying the RLCOACH_PRESET hyperparameter.
        """
        return 'preset-macroeconomic-clippedppo'

    def map_hyperparameter(self, name, value):
        """Here we configure some shortcut names for hyperparameters that we expect to use frequently.
        Essentially anything in the preset file can be overridden through a hyperparameter with a name
        like "rl.agent_params.algorithm.etc".
        """
        # maps from alias (key) to fully qualified coach parameter (value)
        mapping = {
            "discount": "rl.agent_params.algorithm.discount",
            "evaluation_episodes": "rl.evaluation_steps:EnvironmentEpisodes",
            "improve_steps": "rl.improve_steps:TrainingSteps"
    

## Train the RL model using the Python SDK Script mode

If you are using local mode, the training will run on the notebook instance. When using SageMaker for training, you can select a GPU or CPU instance. The RLEstimator is used for training RL jobs. 

1. Specify the source directory where the environment, presets and training code is uploaded.
2. Specify the entry point as the training code 
3. Specify the choice of RL toolkit and framework. This automatically resolves to the ECR path for the RL Container. 
4. Define the training parameters such as the instance count, job name, S3 path for output and job name. 
5. Specify the hyperparameters for the RL agent algorithm. The `RLCOACH_PRESET` can be used to specify the RL agent algorithm you want to use. 
6. [Optional] Choose the metrics that you are interested in capturing in your logs. These can also be visualized in CloudWatch and SageMaker Notebooks. The metrics are defined using regular expression matching.


In [10]:
if local_mode:
    instance_type = 'local'
else:
    instance_type = "ml.m4.xlarge"
    
estimator = RLEstimator(source_dir='src',
                      entry_point="train-coach.py",
                      dependencies=["common/sagemaker_rl"],
                      toolkit=RLToolkit.COACH,
                      toolkit_version='0.11.0',
                      framework=RLFramework.TENSORFLOW,
                      role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      output_path=s3_output_path,
                      base_job_name=job_name_prefix,
                      hyperparameters = {
                          "RLCOACH_PRESET" : "preset-macroeconomic-clippedppo",
                          "rl.agent_params.algorithm.discount": 0.97,
                          "rl.evaluation_steps:EnvironmentEpisodes": 5
                      }
                    )
# takes ~15min
# The log may show KL divergence=[0.]. This is expected because the divergences were not necessarily required for 
# Clipped PPO. By default they are not calculated for computational efficiency.
estimator.fit()

2020-05-13 12:06:49 Starting - Starting the training job...
2020-05-13 12:06:51 Starting - Launching requested ML instances...
2020-05-13 12:07:47 Starting - Preparing the instances for training......
2020-05-13 12:08:35 Downloading - Downloading input data...
2020-05-13 12:09:15 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-05-13 12:09:16,119 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-05-13 12:09:16,122 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-05-13 12:09:16,234 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-05-13 12:09:16,247 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {
        "sagemaker_estimator": "RLEstimator"
    },
    "channel_input_dirs": {},
    "curr

Heatup> Name=main_level/agent, Worker=0, Episode=35, Total reward=-0.69, Steps=105, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=36, Total reward=-0.69, Steps=108, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=37, Total reward=-0.56, Steps=111, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=38, Total reward=-0.72, Steps=114, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=39, Total reward=-0.55, Steps=117, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=40, Total reward=-1.13, Steps=120, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=41, Total reward=-0.91, Steps=123, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=42, Total reward=-0.73, Steps=126, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=43, Total reward=-0.79, Steps=129, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=44, To

Heatup> Name=main_level/agent, Worker=0, Episode=393, Total reward=-0.79, Steps=1179, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=394, Total reward=-0.55, Steps=1182, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=395, Total reward=-0.44, Steps=1185, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=396, Total reward=-0.9, Steps=1188, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=397, Total reward=-0.49, Steps=1191, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=398, Total reward=-0.56, Steps=1194, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=399, Total reward=-0.54, Steps=1197, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=400, Total reward=-0.48, Steps=1200, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=401, Total reward=-0.61, Steps=1203, Training iteration=0
Heatup> Name=main_level/agent, Worker=

Heatup> Name=main_level/agent, Worker=0, Episode=626, Total reward=-0.51, Steps=1878, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=627, Total reward=-0.62, Steps=1881, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=628, Total reward=-0.84, Steps=1884, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=629, Total reward=-0.61, Steps=1887, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=630, Total reward=-0.71, Steps=1890, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=631, Total reward=-2.0, Steps=1893, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=632, Total reward=-0.6, Steps=1896, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=633, Total reward=-0.58, Steps=1899, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=634, Total reward=-1.01, Steps=1902, Training iteration=0
Heatup> Name=main_level/agent, Worker=0

Heatup> Name=main_level/agent, Worker=0, Episode=850, Total reward=-0.75, Steps=2550, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=851, Total reward=-0.51, Steps=2553, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=852, Total reward=-0.64, Steps=2556, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=853, Total reward=-0.98, Steps=2559, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=854, Total reward=-0.62, Steps=2562, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=855, Total reward=-0.76, Steps=2565, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=856, Total reward=-0.64, Steps=2568, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=857, Total reward=-1.05, Steps=2571, Training iteration=0
Heatup> Name=main_level/agent, Worker=0, Episode=858, Total reward=-0.89, Steps=2574, Training iteration=0
Heatup> Name=main_level/agent, Worker

...

Training> Name=main_level/agent, Worker=0, Episode=19912, Total reward=-0.54, Steps=59784, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=19913, Total reward=-0.66, Steps=59787, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=19914, Total reward=-0.67, Steps=59790, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=19915, Total reward=-0.57, Steps=59793, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=19916, Total reward=-0.47, Steps=59796, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=19917, Total reward=-0.41, Steps=59799, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=19918, Total reward=-0.68, Steps=59802, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=19919, Total reward=-0.41, Steps=59805, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=19920, Total reward=-0.52, Steps=59808, Train

Training> Name=main_level/agent, Worker=0, Episode=20012, Total reward=-0.52, Steps=60084, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20013, Total reward=-0.48, Steps=60087, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20014, Total reward=-0.41, Steps=60090, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20015, Total reward=-0.4, Steps=60093, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20016, Total reward=-0.45, Steps=60096, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20017, Total reward=-0.5, Steps=60099, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20018, Total reward=-0.39, Steps=60102, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20019, Total reward=-0.47, Steps=60105, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20020, Total reward=-0.53, Steps=60108, Trainin

Training> Name=main_level/agent, Worker=0, Episode=20160, Total reward=-0.49, Steps=60528, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20161, Total reward=-0.46, Steps=60531, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20162, Total reward=-0.46, Steps=60534, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20163, Total reward=-0.52, Steps=60537, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20164, Total reward=-0.64, Steps=60540, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20165, Total reward=-0.54, Steps=60543, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20166, Total reward=-0.49, Steps=60546, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20167, Total reward=-0.48, Steps=60549, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20168, Total reward=-0.49, Steps=60552, Train

Training> Name=main_level/agent, Worker=0, Episode=20261, Total reward=-0.46, Steps=60831, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20262, Total reward=-0.39, Steps=60834, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20263, Total reward=-0.51, Steps=60837, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20264, Total reward=-0.44, Steps=60840, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20265, Total reward=-0.64, Steps=60843, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20266, Total reward=-0.52, Steps=60846, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20267, Total reward=-0.61, Steps=60849, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20268, Total reward=-0.56, Steps=60852, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20269, Total reward=-0.49, Steps=60855, Train

Training> Name=main_level/agent, Worker=0, Episode=20362, Total reward=-0.6, Steps=61134, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20363, Total reward=-0.63, Steps=61137, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20364, Total reward=-0.58, Steps=61140, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20365, Total reward=-0.47, Steps=61143, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20366, Total reward=-0.64, Steps=61146, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20367, Total reward=-0.44, Steps=61149, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20368, Total reward=-0.47, Steps=61152, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20369, Total reward=-0.61, Steps=61155, Training iteration=25
Training> Name=main_level/agent, Worker=0, Episode=20370, Total reward=-0.45, Steps=61158, Traini

## Store intermediate training output and model checkpoints 

The output from the training job above is either stored in a local directory (`local` mode) or on S3 (`SageMaker`) mode.


In [11]:
%%time

job_name=estimator._current_job_name
print("Job name: {}".format(job_name))

s3_url = "s3://{}/{}".format(s3_bucket,job_name)

if local_mode:
    output_tar_key = "{}/output.tar.gz".format(job_name)
else:
    output_tar_key = "{}/output/output.tar.gz".format(job_name)

intermediate_folder_key = "{}/output/intermediate/".format(job_name)
output_url = "s3://{}/{}".format(s3_bucket, output_tar_key)
intermediate_url = "s3://{}/{}".format(s3_bucket, intermediate_folder_key)

print("S3 job path: {}".format(s3_url))
print("Output.tar.gz location: {}".format(output_url))
print("Intermediate folder path: {}".format(intermediate_url))
    
tmp_dir = "/tmp/{}".format(job_name)
os.system("mkdir {}".format(tmp_dir))
print("Create local folder {}".format(tmp_dir))

Job name: rl-macroeconomic-2020-05-13-12-06-48-952
S3 job path: s3://sagemaker-eu-central-1-415877977751/rl-macroeconomic-2020-05-13-12-06-48-952
Output.tar.gz location: s3://sagemaker-eu-central-1-415877977751/rl-macroeconomic-2020-05-13-12-06-48-952/output/output.tar.gz
Intermediate folder path: s3://sagemaker-eu-central-1-415877977751/rl-macroeconomic-2020-05-13-12-06-48-952/output/intermediate/
Create local folder /tmp/rl-macroeconomic-2020-05-13-12-06-48-952
CPU times: user 0 ns, sys: 8.17 ms, total: 8.17 ms
Wall time: 6.97 ms


In [12]:
%%time

wait_for_s3_object(s3_bucket, output_tar_key, tmp_dir)  

if not os.path.isfile("{}/output.tar.gz".format(tmp_dir)):
    raise FileNotFoundError("File output.tar.gz not found")
os.system("tar -xvzf {}/output.tar.gz -C {}".format(tmp_dir, tmp_dir))
if not local_mode:
    os.system("aws s3 cp --recursive {} {}".format(intermediate_url, tmp_dir))
if not os.path.isfile("{}/output.tar.gz".format(tmp_dir)):
    raise FileNotFoundError("File output.tar.gz not found")
os.system("tar -xvzf {}/output.tar.gz -C {}".format(tmp_dir, tmp_dir))
print("Copied output files to {}".format(tmp_dir))

if local_mode:
    checkpoint_dir = "{}/data/checkpoint".format(tmp_dir)
    info_dir = "{}/data/".format(tmp_dir)
else:
    checkpoint_dir = "{}/checkpoint".format(tmp_dir)
    info_dir = "{}/".format(tmp_dir)

print("Checkpoint directory {}".format(checkpoint_dir))
print("info directory {}".format(info_dir))

Waiting for s3://sagemaker-eu-central-1-415877977751/rl-macroeconomic-2020-05-13-12-06-48-952/output/output.tar.gz...
Copied output files to /tmp/rl-macroeconomic-2020-05-13-12-06-48-952
Checkpoint directory /tmp/rl-macroeconomic-2020-05-13-12-06-48-952/checkpoint
info directory /tmp/rl-macroeconomic-2020-05-13-12-06-48-952/
CPU times: user 209 ms, sys: 84.3 ms, total: 294 ms
Wall time: 3.97 s


## Visualization

### Plot rate of learning

We can view the rewards during training using the code below. This visualization helps us understand how the performance of the model represented as the reward has improved over time. For the consideration of training time, we restict the episodes number. If you see the final reward (average logarithmic cumulated return) is still below zero, try a larger training steps. The number of steps can be configured in the preset file.

In [ ]:
%matplotlib inline
import pandas as pd

csv_file_name = "worker_0.simple_rl_graph.main_level.main_level.agent_0.csv"
key = os.path.join(intermediate_folder_key, csv_file_name)
wait_for_s3_object(s3_bucket, key, tmp_dir)

csv_file = "{}/{}".format(tmp_dir, csv_file_name)
df = pd.read_csv(csv_file)
df = df.dropna(subset=['Training Reward'])
# print(list(df))
x_axis = 'Episode #'
y_axis = 'Training Reward'

plt = df.plot(x=x_axis,y=y_axis, figsize=(12,5), legend=True, style='b-')
plt.set_ylabel(y_axis);
plt.set_xlabel(x_axis);

### Visualize the portfolio value

We use result of the last evaluation phase as an example to visualize the portfolio value. The following figure demonstrates reward vs date. Sharpe ratio and maximum drawdown are also calculated to help readers understand the return of an investment compared to its risk. 

In [ ]:
info_dir

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 300)
info = info_dir + 'macroeconomic.csv'
df_info = pd.read_csv(info)
# df_info[df_info['state'] == '(0.0, 2.05, 1.2446216995824309)']
df_info
#df_info['date'] = pd.to_datetime(df_info['date'], format='%Y-%m-%d')
#df_info.set_index('date', inplace=True)
#mdd = max_drawdown(df_info.rate_of_return + 1)
#sharpe_ratio = sharpe(df_info.rate_of_return)
#title = 'max_drawdown={: 2.2%} sharpe_ratio={: 2.4f}'.format(mdd, sharpe_ratio)
#df_info[["portfolio_value", "market_value"]].plot(title=title, fig=plt.gcf(), rot=30)

## Load the checkpointed models for evaluation

Checkpointed data from the previously trained models will be passed on for evaluation / inference in the `checkpoint` channel. In `local` mode, we can simply use the local directory, whereas in the `SageMaker` mode, it needs to be moved to S3 first.

Since TensorFlow stores ckeckpoint file containes absolute paths from when they were generated (see [issue](https://github.com/tensorflow/tensorflow/issues/9146)), we need to replace the absolute paths to relative paths. This is implemented within `evaluate-coach.py`


In [ ]:
%%time

if local_mode:
    checkpoint_path = 'file://{}'.format(checkpoint_dir)
    print("Local checkpoint file path: {}".format(checkpoint_path))
else:
    checkpoint_path = "s3://{}/{}/checkpoint/".format(s3_bucket, job_name)
    if not os.listdir(checkpoint_dir):
        raise FileNotFoundError("Checkpoint files not found under the path")
    os.system("aws s3 cp --recursive {} {}".format(checkpoint_dir, checkpoint_path))
    print("S3 checkpoint file path: {}".format(checkpoint_path))

### Run the evaluation step

Use the checkpointed model to run the evaluation step. 


In [ ]:
%%time

estimator_eval = RLEstimator(role=role,
                      source_dir='src/',
                      dependencies=["common/sagemaker_rl"],
                      toolkit=RLToolkit.COACH,
                      toolkit_version='0.11.0',
                      framework=RLFramework.MXNET,
                      entry_point="evaluate-coach.py",
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      hyperparameters = {
                          "evaluate_steps": 731*2 # evaluate on 2 episodes
                      }
                    )
estimator_eval.fit({'checkpoint': checkpoint_path})

## Risk Disclaimer (for live-trading)

This notebook is for educational purposes only. Past trading performance does not guarantee future performance. The loss in trading can be substantial, and therefore 
**investors should use all trading strategies at their own risk**.